In [3]:
!yolo task=detect mode=train model=yolo11s.pt data={"/home/kishore-ravishankar/Downloads/dasda/eyes.v1i.yolov11/data.yaml"} epochs= 30  imgsz=640 plots=True batch=16


Ultralytics 8.3.63 🚀 Python-3.12.3 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7933MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/home/kishore-ravishankar/Downloads/dasda/eyes.v1i.yolov11/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.cuda.get_device_name(0))  # Prints the name of the GPU


True
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
!nvidia-smi


Sat Jan 18 20:11:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   41C    P0             15W /   80W |       8MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install dlib

  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py) ... done
|

In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance
from ultralytics import YOLO
import torch
import mediapipe as mp

# Load the trained YOLOv11 model (using GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO("runs/detect/train9/weights/best.pt")  # Load your custom trained model
model.to(device)  # Move the model to GPU if available

# Initialize MediaPipe face mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.3, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    # Calculate the Euclidean distances between vertical eye landmarks
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    
    # Calculate the Euclidean distance between horizontal eye landmarks
    C = distance.euclidean(eye[0], eye[3])
    
    # Return the Eye Aspect Ratio (EAR)
    ear = (A + B) / (2.0 * C)
    return ear

# Thresholds for EAR to detect blinking and fatigue
EAR_THRESHOLD = 0.3
BLINK_CONSEC_FRAMES = 15

# Setup webcam capture
cap = cv2.VideoCapture(0)

frame_count = 0
drowsiness_detected = False

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run the YOLO model on the current frame
    # Convert frame to RGB for YOLO processing
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb)  # Get predictions using the YOLO model
    
    # The results are now contained in the `results` object (which is a list of Results objects)
    # Access the first image's result
    result = results[0]  # Get the result from the first image in the batch
    
    # Extract the predicted labels and their classes
    boxes = result.boxes  # This contains the bounding box information
    class_ids = boxes.cls.cpu().numpy().astype(int)  # Predicted class IDs (use `.cls` for class IDs)
    labels = result.names  # Map class IDs to class labels (e.g., 'Awake', 'Drowsy')
    
    # Check if the person is Awake or Drowsy based on YOLO output
    for i in range(len(boxes)):
        label = labels[class_ids[i]]
        if label == 'Drowsy':
            drowsiness_detected = True
            cv2.putText(frame, "Drowsy", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            drowsiness_detected = False
            cv2.putText(frame, "Awake", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get face landmarks using MediaPipe
    results_mesh = face_mesh.process(rgb_frame)
    
    if results_mesh.multi_face_landmarks:
        for face_landmarks in results_mesh.multi_face_landmarks:
            # Get the left and right eye landmarks (indices are based on the 468 landmarks of MediaPipe)
            left_eye = [face_landmarks.landmark[i] for i in range(33, 133)]
            right_eye = [face_landmarks.landmark[i] for i in range(133, 233)]
            
            # Convert MediaPipe landmarks to pixel coordinates
            h, w, _ = frame.shape
            left_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in left_eye]
            right_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in right_eye]
            
            # Calculate EAR for both eyes
            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            
            # Average EAR for both eyes
            ear = (left_ear + right_ear) / 2.0
            
            # Check for fatigue based on EAR
            if ear < EAR_THRESHOLD:
                frame_count += 1
                if frame_count >= BLINK_CONSEC_FRAMES:
                    drowsiness_detected = True
                    cv2.putText(frame, "Fatigue Detected", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                frame_count = 0
        
            # Draw the landmarks on the face
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)
        
    # Display the resulting frame
    if drowsiness_detected:
        cv2.putText(frame, "Warning: You are Drowsy!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show the frame with the annotations
    cv2.imshow('Drowsiness Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


E0000 00:00:1737297117.034419   20700 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737297117.037413   20700 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1737297118.266828   20700 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737297118.268701   20837 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.3), renderer: Mesa Intel(R) Graphics (RPL-S)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737297118.271420   20810 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737297118.279342   20817 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disa


0: 480x640 (no detections), 27.1ms
Speed: 2.2ms preprocess, 27.1ms inference, 23.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.6ms
Speed: 1.6ms preprocess, 4.6ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 1.0ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 1.0ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.7ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.7ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.7ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)


W0000 00:00:1737297119.550999   20816 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.



0: 480x640 (no detections), 3.9ms
Speed: 0.7ms preprocess, 3.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.8ms
Speed: 0.6ms preprocess, 3.8ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.6ms preprocess, 3.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.6ms preprocess, 3.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.8ms
Speed: 0.6ms preprocess, 3.8ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.0ms
Speed: 1.1ms preprocess, 4.0ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.3ms 

In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance
from ultralytics import YOLO
import torch
import mediapipe as mp
import time

# Load the trained YOLOv11 model (using GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO("runs/detect/train9/weights/best.pt")  # Load your custom trained model
model.to(device)  # Move the model to GPU if available

# Initialize MediaPipe face mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    # Calculate the Euclidean distances between vertical eye landmarks
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    
    # Calculate the Euclidean distance between horizontal eye landmarks
    C = distance.euclidean(eye[0], eye[3])
    
    # Return the Eye Aspect Ratio (EAR)
    ear = (A + B) / (2.0 * C)
    return ear

# Thresholds for EAR to detect blinking and fatigue
EAR_THRESHOLD = 0.3
BLINK_CONSEC_FRAMES = 15
HEAD_DOWN_THRESHOLD = 0.5  # Relative position threshold for head-down detection

# Setup webcam capture
cap = cv2.VideoCapture(0)

frame_count = 0
drowsiness_detected = False
blink_count = 0
eye_closed_time = 0  # Total time in seconds the eyes are closed
head_down_count = 0
head_down_start_time = None  # Track when the head-down state starts
head_down_duration = 0  # Track how long head has been down
head_down_started = False  # Flag to track if the head down detection has started

# To track time when eyes are closed
last_eye_closed_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run the YOLO model on the current frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb)  # Get predictions using the YOLO model
    
    # The results are now contained in the `results` object (which is a list of Results objects)
    # Access the first image's result
    result = results[0]  # Get the result from the first image in the batch
    
    # Extract the predicted labels and their classes
    boxes = result.boxes  # This contains the bounding box information
    class_ids = boxes.cls.cpu().numpy().astype(int)  # Predicted class IDs (use `.cls` for class IDs)
    labels = result.names  # Map class IDs to class labels (e.g., 'Awake', 'Drowsy')
    
    # Check if the person is Awake or Drowsy based on YOLO output
    for i in range(len(boxes)):
        label = labels[class_ids[i]]
        if label == 'Drowsy':
            drowsiness_detected = True
            cv2.putText(frame, "Drowsy", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            drowsiness_detected = False
            cv2.putText(frame, "Awake", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get face landmarks using MediaPipe
    results_mesh = face_mesh.process(rgb_frame)
    
    if results_mesh.multi_face_landmarks:
        head_down_started = False  # Reset flag when face is detected
        
        for face_landmarks in results_mesh.multi_face_landmarks:
            # Get the left and right eye landmarks (indices are based on the 468 landmarks of MediaPipe)
            left_eye = [face_landmarks.landmark[i] for i in range(33, 133)]
            right_eye = [face_landmarks.landmark[i] for i in range(133, 233)]
            
            # Convert MediaPipe landmarks to pixel coordinates
            h, w, _ = frame.shape
            left_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in left_eye]
            right_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in right_eye]
            
            # Calculate EAR for both eyes
            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            
            # Average EAR for both eyes
            ear = (left_ear + right_ear) / 2.0
            
            # Check for fatigue based on EAR
            if ear < EAR_THRESHOLD:
                frame_count += 1
                if frame_count >= BLINK_CONSEC_FRAMES:
                    blink_count += 1
                    cv2.putText(frame, "Fatigue Detected", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
                # Track how long eyes are closed
                if last_eye_closed_time is None:
                    last_eye_closed_time = time.time()
                else:
                    eye_closed_time = int(time.time() - last_eye_closed_time)
            else:
                frame_count = 0
                last_eye_closed_time = None
        
            # Draw the landmarks on the face
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)

            # Detect head down (assuming the face orientation in the frame is used)
            head_pos_y = face_landmarks.landmark[1].y  # Use a landmark near the center of the face
            if head_pos_y > HEAD_DOWN_THRESHOLD:  # If the head position is below a threshold
                cv2.putText(frame, "Head Down Detected", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    else:
        # No face detected, assume head down and start counting
        if not head_down_started:
            head_down_started = True
            head_down_start_time = time.time()  # Record the time when no face is detected
    
    # If no face is detected, calculate how long the head has been down
    if head_down_started:
        head_down_duration = int(time.time() - head_down_start_time)
        cv2.putText(frame, f"Head Down for: {head_down_duration}s", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Display real-time statistics (seconds closed, blink count, head down count)
    cv2.putText(frame, f"Eyes Closed: {eye_closed_time}s", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Blink Count: {blink_count}", (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Head Down Count: {head_down_count}", (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

    # Show the frame with the annotations
    cv2.imshow('Drowsiness Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
from scipy.spatial import distance
from ultralytics import YOLO
import torch
import mediapipe as mp
import time

# Load the trained YOLOv5 model (using GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO("runs/detect/train9/weights/best.pt")  # Load your custom trained model
model.to(device)  # Move the model to GPU if available

# Initialize MediaPipe face mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    # Calculate the Euclidean distances between vertical eye landmarks
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    
    # Calculate the Euclidean distance between horizontal eye landmarks
    C = distance.euclidean(eye[0], eye[3])
    
    # Return the Eye Aspect Ratio (EAR)
    ear = (A + B) / (2.0 * C)
    return ear

# Thresholds for EAR to detect blinking and fatigue
EAR_THRESHOLD = 0.3
BLINK_CONSEC_FRAMES = 15
HEAD_DOWN_THRESHOLD = 0.5  # Relative position threshold for head-down detection

# Setup webcam capture
cap = cv2.VideoCapture(0)

frame_count = 0
drowsiness_detected = False
blink_count = 0
eye_closed_time = 0  # Total time in seconds the eyes are closed
head_down_count = 0
head_down_start_time = None  # Track when the head-down state starts
head_down_duration = 0  # Track how long head has been down
head_down_started = False  # Flag to track if the head down detection has started

# To track time when eyes are closed
last_eye_closed_time = None

# Variables to track head down event duration and count
head_down_event_started = False
head_down_event_duration = 0  # Duration the head has been down (in seconds)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run the YOLO model on the current frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb)  # Get predictions using the YOLO model
    
    # The results are now contained in the `results` object (which is a list of Results objects)
    # Access the first image's result
    result = results[0]  # Get the result from the first image in the batch
    
    # Extract the predicted labels and their classes
    boxes = result.boxes  # This contains the bounding box information
    class_ids = boxes.cls.cpu().numpy().astype(int)  # Predicted class IDs (use `.cls` for class IDs)
    labels = result.names  # Map class IDs to class labels (e.g., 'Awake', 'Drowsy')
    
    # Check if the person is Awake or Drowsy based on YOLO output
    for i in range(len(boxes)):
        label = labels[class_ids[i]]
        if label == 'Drowsy':
            drowsiness_detected = True
            cv2.putText(frame, "Drowsy", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            drowsiness_detected = False
            cv2.putText(frame, "Awake", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get face landmarks using MediaPipe
    results_mesh = face_mesh.process(rgb_frame)
    
    if results_mesh.multi_face_landmarks:
        head_down_started = False  # Reset flag when face is detected
        
        for face_landmarks in results_mesh.multi_face_landmarks:
            # Get the left and right eye landmarks (indices are based on the 468 landmarks of MediaPipe)
            left_eye = [face_landmarks.landmark[i] for i in range(33, 133)]
            right_eye = [face_landmarks.landmark[i] for i in range(133, 233)]
            
            # Convert MediaPipe landmarks to pixel coordinates
            h, w, _ = frame.shape
            left_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in left_eye]
            right_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in right_eye]
            
            # Calculate EAR for both eyes
            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            
            # Average EAR for both eyes
            ear = (left_ear + right_ear) / 2.0
            
            # Check for fatigue based on EAR
            if ear < EAR_THRESHOLD:
                frame_count += 1
                if frame_count >= BLINK_CONSEC_FRAMES:
                    blink_count += 1
                    cv2.putText(frame, "Fatigue Detected", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
                # Track how long eyes are closed
                if last_eye_closed_time is None:
                    last_eye_closed_time = time.time()
                else:
                    eye_closed_time = int(time.time() - last_eye_closed_time)
            else:
                frame_count = 0
                last_eye_closed_time = None
        
            # Draw the landmarks on the face
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)

            # Detect head down (assuming the face orientation in the frame is used)
            head_pos_y = face_landmarks.landmark[1].y  # Use a landmark near the center of the face
            if head_pos_y > HEAD_DOWN_THRESHOLD:  # If the head position is below a threshold
                cv2.putText(frame, "Head Down Detected", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    else:
        # No face detected, assume head down and start counting
        if not head_down_event_started:
            head_down_event_started = True
            head_down_start_time = time.time()  # Record the time when no face is detected
    
    # If no face is detected, calculate how long the head has been down
    if head_down_event_started:
        head_down_event_duration = int(time.time() - head_down_start_time)
        cv2.putText(frame, f"Head Down for: {head_down_event_duration}s", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Once the head has been down for a threshold time, count it as a head down event
        if head_down_event_duration >= 5:  # Adjust threshold to suit your needs
            head_down_count += 1
            head_down_event_started = False  # Reset head down event tracking after counting
    
    # Display real-time statistics (seconds closed, blink count, head down count)
    cv2.putText(frame, f"Eyes Closed: {eye_closed_time}s", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Blink Count: {blink_count}", (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Head Down Count: {head_down_count}", (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

    # Show the frame with the annotations
    cv2.imshow('Drowsiness Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


E0000 00:00:1737299401.309520   24254 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737299401.312500   24254 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1737299404.011456   24254 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737299404.013140   25189 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.3), renderer: Mesa Intel(R) Graphics (RPL-S)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737299404.015541   25161 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737299404.022484   25182 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disa


0: 480x640 (no detections), 29.6ms
Speed: 2.0ms preprocess, 29.6ms inference, 28.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 1.0ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.9ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.8ms
Speed: 1.5ms preprocess, 3.8ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.7ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.8ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)


W0000 00:00:1737299405.269077   25169 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.



0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.7ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.8ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms 

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance
from ultralytics import YOLO
import torch
import mediapipe as mp
import time
import tkinter as tk
from tkinter import messagebox

# Load the trained YOLOv5 model (using GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO("runs/detect/train9/weights/best.pt")  # Load your custom trained model
model.to(device)  # Move the model to GPU if available

# Initialize MediaPipe face mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    # Calculate the Euclidean distances between vertical eye landmarks
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    
    # Calculate the Euclidean distance between horizontal eye landmarks
    C = distance.euclidean(eye[0], eye[3])
    
    # Return the Eye Aspect Ratio (EAR)
    ear = (A + B) / (2.0 * C)
    return ear

# Thresholds for EAR to detect blinking and fatigue
EAR_THRESHOLD = 0.3
BLINK_CONSEC_FRAMES = 15
HEAD_DOWN_THRESHOLD = 0.45  # Head tilt threshold for detecting head down (in radians)

# Setup webcam capture
cap = cv2.VideoCapture(0)

frame_count = 0
drowsiness_detected = False
blink_count = 0
eye_closed_time = 0  # Total time in seconds the eyes are closed
head_down_count = 0
head_down_start_time = None  # Track when the head-down state starts
head_down_duration = 0  # Track how long head has been down
head_down_started = False  # Flag to track if the head down detection has started

# To track time when eyes are closed
last_eye_closed_time = None

# Variables to track head down event duration and count
head_down_event_started = False
head_down_event_duration = 0  # Duration the head has been down (in seconds)

# Function to show tkinter pop-up
def show_popup(message, title="Alert"):
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    messagebox.showinfo(title, message)
    root.destroy()  # Destroy the Tkinter root window

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run the YOLO model on the current frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb)  # Get predictions using the YOLO model
    
    # The results are now contained in the `results` object (which is a list of Results objects)
    # Access the first image's result
    result = results[0]  # Get the result from the first image in the batch
    
    # Extract the predicted labels and their classes
    boxes = result.boxes  # This contains the bounding box information
    class_ids = boxes.cls.cpu().numpy().astype(int)  # Predicted class IDs (use `.cls` for class IDs)
    labels = result.names  # Map class IDs to class labels (e.g., 'Awake', 'Drowsy')
    
    # Check if the person is Awake or Drowsy based on YOLO output
    for i in range(len(boxes)):
        label = labels[class_ids[i]]
        if label == 'Drowsy':
            drowsiness_detected = True
            cv2.putText(frame, "Drowsy", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            drowsiness_detected = False
            cv2.putText(frame, "Awake", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get face landmarks using MediaPipe
    results_mesh = face_mesh.process(rgb_frame)
    
    if results_mesh.multi_face_landmarks:
        head_down_started = False  # Reset flag when face is detected
        
        for face_landmarks in results_mesh.multi_face_landmarks:
            # Get the left and right eye landmarks (indices are based on the 468 landmarks of MediaPipe)
            left_eye = [face_landmarks.landmark[i] for i in range(33, 133)]
            right_eye = [face_landmarks.landmark[i] for i in range(133, 233)]
            
            # Convert MediaPipe landmarks to pixel coordinates
            h, w, _ = frame.shape
            left_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in left_eye]
            right_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in right_eye]
            
            # Calculate EAR for both eyes
            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            
            # Average EAR for both eyes
            ear = (left_ear + right_ear) / 2.0
            
            # Check for fatigue based on EAR
            if ear < EAR_THRESHOLD:
                frame_count += 1
                if frame_count >= BLINK_CONSEC_FRAMES:
                    blink_count += 1
                    cv2.putText(frame, "Fatigue Detected", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
                # Track how long eyes are closed
                if last_eye_closed_time is None:
                    last_eye_closed_time = time.time()
                else:
                    eye_closed_time = int(time.time() - last_eye_closed_time)
            else:
                frame_count = 0
                last_eye_closed_time = None
        
            # Draw the landmarks on the face
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)

            # Detect head down (head tilt detection)
            # Get the angle of the head tilt based on landmarks (for simplicity, using landmarks at index 1 and 10)
            nose_tip = face_landmarks.landmark[1]
            chin = face_landmarks.landmark[152]
            dx = nose_tip.x - chin.x
            dy = nose_tip.y - chin.y
            head_angle = np.arctan2(dy, dx)  # Get angle of the head in radians

            # Check if the head angle is below a threshold to consider it as head down (e.g., 45 degrees or more)
            if abs(head_angle) > np.radians(45):  # Head is down by 45 degrees or more
                head_down_count += 1
                head_down_event_started = True
                head_down_start_time = time.time()
                cv2.putText(frame, "Head Down Detected", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    else:
        # No face detected, assume head down and start counting
        if not head_down_event_started:
            head_down_event_started = True
            head_down_start_time = time.time()  # Record the time when no face is detected
    
    # If no face is detected, calculate how long the head has been down
    if head_down_event_started:
        head_down_event_duration = int(time.time() - head_down_start_time)
        cv2.putText(frame, f"Head Down for: {head_down_event_duration}s", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Once the head has been down for a threshold time, count it as a head down event
        if head_down_event_duration >= 5:  # Adjust threshold to suit your needs
            show_popup(f"Head Down Detected for {head_down_event_duration}s", "Head Down Alert")
            head_down_event_started = False  # Reset head down event tracking after counting
    
    # Display real-time statistics (seconds closed, blink count, head down count)
    cv2.putText(frame, f"Eyes Closed: {eye_closed_time}s", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Blink Count: {blink_count}", (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Head Down Count: {head_down_count}", (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

    # Show the frame with the annotations
    cv2.imshow('Drowsiness Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


E0000 00:00:1737297631.211693   21265 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737297631.215006   21265 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1737297633.999115   21265 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737297634.001340   21403 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.3), renderer: Mesa Intel(R) Graphics (RPL-S)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1737297634.003922   21379 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737297634.008829   21376 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disa


0: 480x640 (no detections), 30.2ms
Speed: 2.0ms preprocess, 30.2ms inference, 76.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 1.1ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.9ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.5ms
Speed: 1.6ms preprocess, 7.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.0ms
Speed: 1.3ms preprocess, 4.0ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.7ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)


W0000 00:00:1737297635.359025   21375 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.



0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.7ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 0.6ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.8ms
Speed: 0.6ms preprocess, 3.8ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.6ms preprocess, 3.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.8ms preprocess, 3.9ms inference, 0.2ms 

In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance
from ultralytics import YOLO
import torch
import mediapipe as mp
import time
import tkinter as tk
from tkinter import messagebox

# Load the trained YOLOv5 model (using GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO("runs/detect/train9/weights/best.pt")  # Load your custom trained model
model.to(device)  # Move the model to GPU if available

# Initialize MediaPipe face mesh model
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    # Calculate the Euclidean distances between vertical eye landmarks
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    
    # Calculate the Euclidean distance between horizontal eye landmarks
    C = distance.euclidean(eye[0], eye[3])
    
    # Return the Eye Aspect Ratio (EAR)
    ear = (A + B) / (2.0 * C)
    return ear

# Thresholds for EAR to detect blinking and fatigue
EAR_THRESHOLD = 0.3
BLINK_CONSEC_FRAMES = 15
HEAD_DOWN_THRESHOLD = 0.45  # Head tilt threshold for detecting head down (in radians)

# Setup webcam capture
cap = cv2.VideoCapture(0)

frame_count = 0
drowsiness_detected = False
blink_count = 0
eye_closed_time = 0  # Total time in seconds the eyes are closed
head_down_count = 0
head_down_start_time = None  # Track when the head-down state starts
head_down_duration = 0  # Track how long head has been down
head_down_started = False  # Flag to track if the head down detection has started

# To track time when eyes are closed
last_eye_closed_time = None

# Variables to track head down event duration and count
head_down_event_started = False
head_down_event_duration = 0  # Duration the head has been down (in seconds)

# Function to show tkinter pop-up
def show_popup(message, title="Alert"):
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    messagebox.showinfo(title, message)
    root.destroy()  # Destroy the Tkinter root window

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Run the YOLO model on the current frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb)  # Get predictions using the YOLO model
    
    # The results are now contained in the `results` object (which is a list of Results objects)
    # Access the first image's result
    result = results[0]  # Get the result from the first image in the batch
    
    # Extract the predicted labels and their classes
    boxes = result.boxes  # This contains the bounding box information
    class_ids = boxes.cls.cpu().numpy().astype(int)  # Predicted class IDs (use `.cls` for class IDs)
    labels = result.names  # Map class IDs to class labels (e.g., 'Awake', 'Drowsy')
    
    # Check if the person is Awake or Drowsy based on YOLO output
    for i in range(len(boxes)):
        label = labels[class_ids[i]]
        if label == 'Drowsy':
            drowsiness_detected = True
            cv2.putText(frame, "Drowsy", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            drowsiness_detected = False
            cv2.putText(frame, "Awake", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get face landmarks using MediaPipe
    results_mesh = face_mesh.process(rgb_frame)
    
    if results_mesh.multi_face_landmarks:
        head_down_started = False  # Reset flag when face is detected
        
        for face_landmarks in results_mesh.multi_face_landmarks:
            # Get the left and right eye landmarks (indices are based on the 468 landmarks of MediaPipe)
            left_eye = [face_landmarks.landmark[i] for i in range(33, 133)]
            right_eye = [face_landmarks.landmark[i] for i in range(133, 233)]
            
            # Convert MediaPipe landmarks to pixel coordinates
            h, w, _ = frame.shape
            left_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in left_eye]
            right_eye = [(int(landmark.x * w), int(landmark.y * h)) for landmark in right_eye]
            
            # Calculate EAR for both eyes
            left_ear = eye_aspect_ratio(left_eye)
            right_ear = eye_aspect_ratio(right_eye)
            
            # Average EAR for both eyes
            ear = (left_ear + right_ear) / 2.0
            
            # Check for fatigue based on EAR
            if ear < EAR_THRESHOLD:
                frame_count += 1
                if frame_count >= BLINK_CONSEC_FRAMES:
                    blink_count += 1
                    cv2.putText(frame, "Fatigue Detected", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                
                # Track how long eyes are closed
                if last_eye_closed_time is None:
                    last_eye_closed_time = time.time()
                else:
                    eye_closed_time = int(time.time() - last_eye_closed_time)
            else:
                frame_count = 0
                last_eye_closed_time = None
        
            # Draw the landmarks on the face
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)

    else:
        # No face detected, assume head down and start counting
        if not head_down_started:
            head_down_started = True
            head_down_start_time = time.time()  # Record the time when no face is detected
    
    # If no face is detected, calculate how long the head has been down
    if not results_mesh.multi_face_landmarks and head_down_started:
        head_down_duration = int(time.time() - head_down_start_time)
        
        # Check if the head is down
        if head_down_duration >= 1:  # Head down for more than 1 second
            cv2.putText(frame, f"Head Down for: {head_down_duration}s", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            head_down_count += 1
            show_popup(f"Head Down Detected for {head_down_duration}s", "Head Down Alert")
        
    # Display real-time statistics (seconds closed, blink count, head down count)
    cv2.putText(frame, f"Eyes Closed: {eye_closed_time}s", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Blink Count: {blink_count}", (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Head Down Count: {head_down_count}", (50, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

    # Show the frame with the annotations
    cv2.imshow('Drowsiness Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1737299161.393773   23191 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1737299161.394467   24229 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.9-0ubuntu0.3), renderer: Mesa Intel(R) Graphics (RPL-S)
W0000 00:00:1737299161.396489   24195 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1737299161.401163   24197 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.



0: 480x640 (no detections), 29.4ms
Speed: 1.1ms preprocess, 29.4ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 1.2ms preprocess, 3.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.3ms
Speed: 1.6ms preprocess, 7.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.1ms
Speed: 1.5ms preprocess, 4.1ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.7ms
Speed: 1.2ms preprocess, 3.7ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.2ms
Speed: 1.1ms preprocess, 7.2ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.3ms
Speed: 1.2ms preprocess, 7.3ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)


W0000 00:00:1737299162.546198   24199 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.



0: 480x640 (no detections), 7.2ms
Speed: 1.2ms preprocess, 7.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.6ms preprocess, 3.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.9ms preprocess, 3.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.7ms preprocess, 3.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.8ms
Speed: 0.7ms preprocess, 3.8ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.9ms preprocess, 3.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.1ms
Speed: 0.6ms preprocess, 4.1ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.8ms
Speed: 1.1ms preprocess, 3.8ms inference, 0.2ms 

In [3]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 12.1 MB/s eta 0:00:0031m12.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 11.9 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 11.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
